# Numerical Optimization (CS215300) Assignment 3
## Introduction
In this assignment, we expect you to be able to solve constrained optimization problem by Scipy library. We want you to apply two algorithms on the following problem as a benchmark, survey the methods used in these libraries, and analyze the behavior of these algorithms.
The library document link: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html

## Task
1. (20%) Solve the following problrem by using **trust-constr** method:
$$\begin{array}{lll}
        \min_{x_1,x_2} & f(x_1,x_2)=-x_1-x_2 \\
        \mbox{s.t. } & -2x_1^4 + 8x_1^3 -8x_1^2 + x_2 - 2 \le 0  \\
         & -4x_1^4 + 32x_1^3 - 88x_1^2 + 96x_1 + x_2 -36 \le 0   \\
         & 0 \le x_1 \le 3 \\
         & 0 \le x_2 \le 4 \\
\end{array}$$
2. (20%) Use **COBYLA** method to solve the same problem.
3. (15%) For the above two algorithms, please include the calculation process in markdown style before your code cells.
4. (5%) Provide the Jacobian and Hessian function in matrix form in markdown style.
5. (40%) In your report, please read the paper cited in the libraries, which gives the details of the algorithms. Write an introduction of the algorithms, and compare their behaviors in this benchmark. You are not necessarily to read the original paper, other resourses (ex. slides from other schools or surveys) are also acceptable. Please include the link or paper name in your reference if you referred to other resources.
6. Rename this notebook file by adding your student ID and upload it to eeclass platform. (ex. hw3_110xxxxxx.ipynb)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from scipy.optimize import Bounds
from scipy.optimize import NonlinearConstraint

### Define objective function

將題目的$f(x_1, x_2)=x_1+x_2$寫入f(x)的函式，也就是return -x[0]-x[1]，或直接return -sum[x]也可。

In [ ]:
def f(x):
    # TODO
    return -x[0]-x[1]

### Define constraint functions and derivatives
Note: Please do not use sparse matrices.

將題目的兩條constraint寫進cons_f(x)的函式，也就是return $\begin{bmatrix}-2x_1^4 + 8x_1^3 -8x_1^2 + x_2 - 2 \\
-4x_1^4 + 32x_1^3 - 88x_1^2 + 96x_1 + x_2 -36
\end{bmatrix}$

再經過手動計算出其一次導數的Jacobian寫進cons_Jacobian(x)的函式，也就是return $\begin{bmatrix} -8x_1^3+24x_1^2 -16x_1 && 1 \\ -16x_1^3+96x_1^2-176x_1+96 && 1\end{bmatrix}$

再經過手動計算出其二次導數的Hessian寫進cons_Hessian(x, v)的函式，也就是return
$v_1
\begin{bmatrix} -24x_1^2+48x_1 -16 && 0 \\ 0 && 0 \end{bmatrix}
+v_2
\begin{bmatrix}  -48x_1^2+192x_1-176 && 0 \\ 0 && 0 \end{bmatrix}$,

其中$v_1$ 和 $v_2$ 是 拉格朗日乘數(Lagrange multipliers)

In [ ]:
# TODO: derive and define the functions
def cons_f(x):
    # TODO
    return [-2.*x[0]**4 + 8.*x[0]**3 - 8.*x[0]**2 + x[1] - 2, -4.*x[0]**4 + 32.*x[0]**3 - 88.*x[0]**2 + 96.*x[0] + x[1] - 36]
    
def cons_Jacobian(x):
    # TODO
    return np.array([[-8.*x[0]**3 + 24.*x[0]**2 - 16.*x[0], 1], [-16.*x[0]**3 + 96.*x[0]**2 - 176.*x[0] + 96, 1]])

def cons_Hessian(x, v):
    # TODO
    return v[0]*np.array([[-24.*x[0]**2 + 48.*x[0] - 16, 0], [0, 0]]) + v[1]*np.array([[-48.*x[0]**2 + 192.*x[0] - 176, 0], [0, 0]])

# TODO: Insert the functions above into a NonlinearConstraint obeject
nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, 0, jac=cons_Jacobian, hess=cons_Hessian)

參考自：https://stackoverflow.com/questions/69839367/nonlinearconstraints-in-scipy-optimize 和 https://scipy.github.io/devdocs/tutorial/optimize.html#trust-region-constrained-algorithm-method-trust-constr

### Define the bounds 

使用Bounds函數建立$x_1$和$x_2$的邊界，而在Bounds函數上下界限的排列中，經過網路查詢好像有將
$0 \le x_1 \le 3 \\\\
0 \le x_2 \le 4 $，

寫作Bounds([0, 0], [3, 4])和Bounds([0, 3], [0, 4])的兩個版本，但我最終選擇採信下方參考資料來源的版本。

In [ ]:
# TODO: define the bounds
bounds = Bounds([0, 0], [3, 4])

In [ ]:
bounds

Bounds(array([0, 0]), array([3, 4]))

參考自：https://scipy.github.io/devdocs/tutorial/optimize.html#trust-region-constrained-algorithm-method-trust-constr

### Call the minimize library

此處先import欲使用到的scipy.optimize函式庫，並執行初始化，

因為trust-constr方法會使用到Jacobian和Hessian matrix，所以照理說兩者應都需設定初始條件，但經過實際測試後，好像只設定Hessian的初始條件也可運作。

最後將所有參數設定進要使用的優化函數中，其參數包含題目function、initial guess、欲使用的trust-constr方法、Hessian matrix的初始條件、$\vec x$ 的邊界限制，和含有Jacobian與Hessian matrix的兩條constraint條件。

而initial guess的部分，我是隨意將其設定在[0, 0]。

In [ ]:
# TODO: define initial point
from scipy.optimize import minimize
def H0(x):
    return np.array([[0, 0], [0, 0]])

x0 = np.array([0, 0])

min_trust_constr = minimize(f, x0, method='trust-constr', hess = H0, bounds = bounds, constraints = [nonlinear_constraint])

### Print out the result you get

最終顯示$\vec x$ 的收斂結果，分別print出$x_1$和$x_2$的位置、$f(\vec x)$的值和兩條constraint的結果，用以觀察是否符合條件。

In [ ]:
print('x = '+str(min_trust_constr.x))
print('f(x) = '+str(f(min_trust_constr.x)))
print('constraint_1 = '+str(-2.*min_trust_constr.x[0]**4 + 8.*min_trust_constr.x[0]**3 - 8.*min_trust_constr.x[0]**2 + min_trust_constr.x[1] - 2))
print('constraint_2 = '+str(-4.*min_trust_constr.x[0]**4 + 32.*min_trust_constr.x[0]**3 - 88.*min_trust_constr.x[0]**2 + 96.*min_trust_constr.x[0] + min_trust_constr.x[1] - 36))
print('---------------------------------------------')
print(min_trust_constr)

x = [0.61157004 3.44188615]
f(x) = -4.053456193162767
constraint_1 = -0.00013076744813034225
constraint_2 = -0.0009031285439107251
---------------------------------------------
 barrier_parameter: 0.00016000000000000007
 barrier_tolerance: 0.00016000000000000007
          cg_niter: 12
      cg_stop_cond: 4
            constr: [array([-0.00013077, -0.00090313]), array([0.61157004, 3.44188615])]
       constr_nfev: [13, 0]
       constr_nhev: [17, 0]
       constr_njev: [12, 0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 0.028287172317504883
               fun: -4.053456193162767
              grad: array([-1.        , -1.00000001])
               jac: [array([[-2.63859589,  1.        ],
       [20.60958198,  1.        ]]), array([[1., 0.],
       [0., 1.]])]
   lagrangian_grad: array([4.46148586e-10, 7.67958032e-09])
           message: '`gtol` termination condition is satisfied.'
            method: 'tr_interior_point'
              nfev: 37
              nhev: 

### Apply COBYLA method

接下來是COBYLA方法的測試，此處先import欲使用到的scipy.optimize函式庫，

並因為COBYLA方法不需使用到Jacobian和Hessian matrix，所以不用進行兩矩陣的初始化，可直接設定優化函數中的參數，

其參數包含題目function、initial guess、欲使用的COBYLA方法、$\vec x$ 的邊界限制，和兩條constraint條件，

另外我還加入了tolerance的設定，因COBYLA方法在scipy.optimize中，預設的tol=none，且catol=0.0002，為確保誤差接受標準與trust-constr方法一致，所以我將其都設定為1e-8。

In [ ]:
# TODO
from scipy.optimize import minimize

x0 = np.array([0, 1])

min_trust_COBYLA = minimize(f, x0, method='COBYLA', bounds = bounds, constraints = [nonlinear_constraint], tol = 1e-8, options={'catol': 1e-8})

最終顯示$\vec x$ 的收斂結果，分別print出$x_1$和$x_2$的位置、$f(\vec x)$的值和兩條constraint的結果，用以觀察是否符合條件。

In [ ]:
print('x = '+str(min_trust_COBYLA.x))
print('f(x) = '+str(f(min_trust_COBYLA.x)))
print('constraint_1 = '+str(-2.*min_trust_COBYLA.x[0]**4 + 8.*min_trust_COBYLA.x[0]**3 - 8.*min_trust_COBYLA.x[0]**2 + min_trust_COBYLA.x[1] - 2))
print('constraint_2 = '+str(-4.*min_trust_COBYLA.x[0]**4 + 32.*min_trust_COBYLA.x[0]**3 - 88.*min_trust_COBYLA.x[0]**2 + 96.*min_trust_COBYLA.x[0] + min_trust_COBYLA.x[1] - 36)+' <--(已小於tol，可視為0)')
print('---------------------------------------------')
print(min_trust_COBYLA)

x = [0.61160327 3.44210458]
f(x) = -4.053707848201464
constraint_1 = -2.220446049250313e-15
constraint_2 = 4.263256414560601e-14 <--(已小於tol，可視為0)
---------------------------------------------
     fun: -4.053707848201464
   maxcv: 4.263256414560601e-14
 message: 'Optimization terminated successfully.'
    nfev: 31
  status: 1
 success: True
       x: array([0.61160327, 3.44210458])


最終結果顯示，其第二條constraint約為$4.2^{-14}$，不符合題目$\le 0$的條件，但因tolerance設定為$1^{-8}$，所以可視為$=0$。

## Report

The trust-region constrained method(信賴域算法)，使用Interior point method來解決優化問題，是一種通用的限制優化方法，無論equality-constraint或inequality-constraints皆可使用，且適合處理大規模問題。

COBYLA是Constrained Optimization BY Linear Approximation的縮寫，是一種線性近似限制優化方法，通過對目標函數和限制條件的線性逼近處理非線性問題。且只使用目標函數，不需要Jacobian(導數)或Hessian(二階導數)。

參考自：https://inf.news/zh-hant/technique/62d5c131511c2aa7668caac0db2c0eaa.html



在本次實作中，

trust-constr在使用initial guess為$[0, 0]$時，$\vec x$ 最終可以收斂在$[0.61157004 3.44188615]$的位置，此時$f(\vec x)=-4.053456193162767$，且$\vec x$ 同時也滿足兩條constraint，皆為$<0$。

而COBYLA在使用initial guess為$[0, 1]$時，$\vec x$ 最終可以收斂在$[0.61160327 3.44210458]$的位置，此時$f(\vec x)=-4.053707848201464$，且$\vec x$ 同時也滿足兩條constraint，constraint_1為$<0$，constraint_2為已小於tolerance的正數，也可視為$=0$。

將兩種方法的結果比較過後，trust-constr找到的找到的$f(\vec x)$較小，也就是$\vec x$ 會較為準確。

$c_1(\overset\rightharpoonup x) = -2x_1^4 + 8x_1^3 -8x_1^2 + x_2 - 2$

$c_2(\overset\rightharpoonup x) = -4x_1^4 + 32x_1^3 - 88x_1^2 + 96x_1 + x_2 -36$

----

$Jacobain = \begin{bmatrix}
\frac {\partial c_1(\overset\rightharpoonup x)}{\partial x_1} && \frac {\partial c_1(\overset\rightharpoonup x)}{\partial x_2} \\ \frac {\partial c_2(\overset\rightharpoonup x)}{\partial x_1} && \frac {\partial c_2(\overset\rightharpoonup x)}{\partial x_2}
\end{bmatrix}
= 
\begin{bmatrix} -8x_1^3+24x_1^2 -16x_1 && 1 \\ -16x_1^3+96x_1^2-176x_1+96 && 1\end{bmatrix}$

----

$Hessian = v_1
\begin{bmatrix}
\frac {\partial^2 c_1(\overset\rightharpoonup x)}{\partial x_1^2} && \frac {\partial^2 c_1(\overset\rightharpoonup x)}{\partial x_1\partial x_2} \\ \frac {\partial^2 c_1(\overset\rightharpoonup x)}{\partial x_1\partial x_2} && \frac {\partial^2 c_1(\overset\rightharpoonup x)}{\partial x_2^2}\end{bmatrix}
+v_2
\begin{bmatrix}
\frac {\partial^2 c_2(\overset\rightharpoonup x)}{\partial x_1^2} && \frac {\partial^2 c_2(\overset\rightharpoonup x)}{\partial x_1\partial x_2} \\ \frac {\partial^2 c_2(\overset\rightharpoonup x)}{\partial x_1\partial x_2} && \frac {\partial^2 c_2(\overset\rightharpoonup x)}{\partial x_2^2}\end{bmatrix}$

=
$v_1
\begin{bmatrix} -24x_1^2+48x_1 -16 && 0 \\ 0 && 0 \end{bmatrix}
+v_2
\begin{bmatrix}  -48x_1^2+192x_1-176 && 0 \\ 0 && 0 \end{bmatrix}$，

其中$v_1$ 和 $v_2$ 是 拉格朗日乘數(Lagrange multipliers).